In [ ]:
print('Setup complete.')

# LAB 14 — Backlog groomer

```


## Original CLI usage

```bash
bash
python lessons/day-05/labs/lab-14/main.py groom --epic EPIC-123
```


In [ ]:
# Environment check (adjust as needed)
import sys, platform
print(f'Python: {sys.version.split()[0]} on {platform.system()}')
# If imports fail, install requirements in your environment.
try:
    import httpx, jinja2, rich, typer, duckdb, pandas, matplotlib  # may be subset per lab
except Exception as e:
    print('Optional imports missing for some labs:', e)


In [ ]:
#!/usr/bin/env python3
"""LAB 14 — Backlog groomer

Split epics, draft AC, detect dependencies, and open tickets.
"""
# typer removed
from rich import print



def groom(epic: str = ...):
    print(f"[yellow]TODO:[/] Propose stories, AC, estimates, and dependencies for {epic}.")


In [ ]:
# Try the functions defined above. For example, if this lab had commands like `draft()` or `apply()`:
# draft()
# apply()
